<a href="https://colab.research.google.com/github/myracheng/155kaggle1/blob/master/dnn_cs155miniproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile 
file_name = "drive/My Drive/train.csv.zip"
  
# # opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipF: 
    # printing all the contents of the zip file 
    zipF.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    data = zipF.extract('train.csv') 
    print('Done!')

file_name = "drive/My Drive/test.csv.zip"
  
# # opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipF: 
    # printing all the contents of the zip file 
    zipF.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    data = zipF.extract('test.csv') 
    print('Done!')

File Name                                             Modified             Size
train.csv                                      2020-02-09 10:27:26     77009507
Extracting all the files now...
Done!
File Name                                             Modified             Size
test.csv                                       2020-02-09 10:27:26     24378147
Extracting all the files now...
Done!


In [0]:
import pandas as pd

In [0]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [0]:
temp = Dataset(csv_file='train.csv',
                  root_dir='data/faces/',
                  transform=transforms.Compose([
                      Rescale(256),
                      RandomCrop(224),
                      ToTensor()
                  ]))

NameError: ignored

In [0]:
train_df.fillna(0)
test_df.fillna(0)

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,bid4,bid5,ask1,ask2,ask3,ask4,ask5,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol
0,592380,5178.4,5178.3,1.0,11.0,12.0,-10,5178.2,5178.0,5177.8,5177.6,5177.4,5178.4,5178.6,5178.8,5179.0,5179.8,3,16,3,1,1,1,4,1,5,2
1,592381,5133.0,5132.9,1.0,0.0,1.0,1,5132.8,5132.6,5132.4,5132.2,5132.0,5133.0,5133.2,5133.8,5134.0,5134.2,1,2,2,1,10,4,8,1,1,1
2,592382,5177.4,5178.0,2.0,0.0,2.0,2,5177.2,5176.8,5176.6,5176.4,5176.0,5178.8,5179.0,5179.2,5179.4,5179.6,8,18,1,1,6,7,4,1,2,5
3,592383,5093.6,5093.9,0.0,2.0,2.0,-1,5093.2,5093.0,5092.8,5092.2,5092.0,5094.6,5094.8,5095.0,5095.2,5095.4,8,5,4,3,1,1,1,11,6,2
4,592384,5189.2,5189.2,0.0,0.0,0.0,0,5188.8,5188.6,5188.4,5188.2,5188.0,5189.6,5190.6,5190.8,5191.0,5191.2,1,5,1,3,3,3,2,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,784234,5150.6,5150.8,0.0,0.0,0.0,0,5150.6,5150.4,5150.2,5150.0,5149.8,5151.0,5151.4,5151.6,5152.0,5152.4,1,5,3,4,2,1,3,1,3,2
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,5121.8,5121.6,5122.8,5123.0,5123.2,5123.4,5123.6,3,1,5,1,20,16,17,4,1,16
191856,784236,5192.8,5192.3,0.0,0.0,0.0,0,5192.0,5191.8,5191.2,5190.4,5190.2,5192.6,5192.8,5193.0,5193.2,5193.4,1,2,1,1,1,1,2,1,2,1
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,5151.2,5151.0,5152.0,5152.2,5152.4,5152.6,5152.8,4,5,6,8,37,7,1,2,1,1


In [0]:
train_df

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,bid4,bid5,ask1,ask2,ask3,ask4,ask5,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,3841.0,3840.4,3842.8,3843.4,3843.6,3843.8,3844.0,8,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,3842.0,3841.8,3843.8,3844.0,3844.4,3844.6,3844.8,7,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,3843.0,3842.8,3844.8,3845.0,3845.4,3845.6,3845.8,3,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,3842.0,3841.4,3843.8,3844.0,3844.4,3844.6,3844.8,10,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,3841.8,3841.4,3843.4,3844.0,3844.6,3844.8,3845.0,14,12,2,2,4,1,3,1,11,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592375,592375,4110.2,4110.3,1.0,1.0,2.0,1,4110.2,4110.0,4109.4,4109.2,4109.0,4110.4,4111.8,4112.0,4112.2,4112.6,2,2,1,3,2,2,3,10,7,7,1
592376,592376,4109.4,4110.5,6.0,5.0,11.0,1,4109.2,4109.0,4108.6,4108.2,4108.0,4111.8,4112.0,4112.2,4112.6,4113.0,3,2,2,6,5,3,9,7,7,5,0
592377,592377,4109.4,4110.5,NaN,NaN,0.0,0,4109.2,4109.0,4108.6,4108.2,4108.0,4111.8,4112.0,4112.2,4112.6,4113.0,3,2,2,6,5,3,9,7,7,5,0
592378,592378,4109.4,4110.5,NaN,NaN,0.0,0,4109.2,4109.0,4108.6,4108.2,4108.0,4111.8,4112.0,4112.2,4112.6,4113.0,3,2,2,6,5,3,9,7,7,5,0


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [0]:
model = nn.Sequential(

    nn.Flatten(),  
    nn.Linear(27, 20),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

train_target = torch.tensor(train_df['y'].values.astype(np.float32),dtype=torch.long)
train = torch.tensor(train_df.drop('y', axis = 1).values.astype(np.float32)) 
train_tensor = torch.utils.data.TensorDataset(train, train_target) 
train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True) 
# print(model)
# Some layers, such as Dropout, behave differently during training

In [0]:
train_tensor.item()

AttributeError: ignored

In [0]:
np.shape(train_target)

torch.Size([592380])

In [0]:

model.train()

for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_fn(output, target)
        print(loss)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))
# Putting layers like Dropout into evaluation mode
model.eval()

test_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllLossBackward>)
tensor(nan, grad_fn=<NllL

KeyboardInterrupt: ignored